# Case Study: PID Controller Tuning using Boosting Models

In this notebook, we will train and evaluate different boosting models (**AdaBoost**, **Gradient Boosting**, and **XGBoost**) to predict the PID controller parameters: `Kp`, `Ki`, and `Kd` based on system inputs. We will simulate the data for the features and targets, split it into training and testing sets, and then use three boosting algorithms for prediction.
The notebook will demonstrate how each model performs and visualize the predictions compared to true values for the three parameters.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
import xgboost as xgb
import matplotlib.pyplot as plt

### Simulating Data for PID Tuning
In this section, we simulate the data for a PID controller. We generate random data for features like `Error`, `Derivative`, `Integral`, and `Setpoint`, and use these to create the target variables `Kp`, `Ki`, and `Kd`. The target variables represent the proportional, integral, and derivative gains, respectively, which are the parameters of the PID controller.


In [ ]:
# Simulate the data for PID controller tuning
np.random.seed(42)

# Generate random data for features: Error (e), Derivative (de/dt), Integral (∫e), Setpoint
n_samples = 1000
error = np.random.uniform(-2, 2, n_samples)  # Error in temperature
derivative = np.random.uniform(-0.5, 0.5, n_samples)  # Rate of change of error
integral = np.cumsum(error)  # Cumulative error over time (integral)
setpoint = 25 + np.random.normal(0, 0.5, n_samples)  # Target setpoint with noise

# Generate PID parameters as target variables (Kp, Ki, Kd)
Kp = 2 + 0.5 * error + 0.1 * derivative
Ki = 0.1 + 0.05 * integral
Kd = 0.02 + 0.01 * derivative

# Create a DataFrame with features and target variables
data = pd.DataFrame({
    'Error': error,
    'Derivative': derivative,
    'Integral': integral,
    'Setpoint': setpoint,
    'Kp': Kp,
    'Ki': Ki,
    'Kd': Kd
})
data.head()

### Feature Scaling and Data Splitting
Before training the models, we scale the features using `StandardScaler` and split the data into training and testing sets. The training set will be used to train the models, while the testing set will be used for evaluation.


In [ ]:
# Features (X) and Target variables (y)
X = data[['Error', 'Derivative', 'Integral', 'Setpoint']]

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, data[['Kp', 'Ki', 'Kd']], test_size=0.2, random_state=42)

### Model Training and Prediction
We will now train three separate models (AdaBoost, Gradient Boosting, and XGBoost) for each of the PID parameters (`Kp`, `Ki`, and `Kd`). After training the models, we will use them to make predictions on the test set.


In [ ]:
# Separate models for Kp, Ki, Kd
# AdaBoost Regressor for Kp
ada_model_kp = AdaBoostRegressor(n_estimators=100, random_state=42)
ada_model_kp.fit(X_train, y_train['Kp'])

# AdaBoost Regressor for Ki
ada_model_ki = AdaBoostRegressor(n_estimators=100, random_state=42)
ada_model_ki.fit(X_train, y_train['Ki'])

# AdaBoost Regressor for Kd
ada_model_kd = AdaBoostRegressor(n_estimators=100, random_state=42)
ada_model_kd.fit(X_train, y_train['Kd'])

# Gradient Boosting Regressor for Kp
gb_model_kp = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model_kp.fit(X_train, y_train['Kp'])

# Gradient Boosting Regressor for Ki
gb_model_ki = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model_ki.fit(X_train, y_train['Ki'])

# Gradient Boosting Regressor for Kd
gb_model_kd = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model_kd.fit(X_train, y_train['Kd'])

# XGBoost Regressor for Kp
xg_model_kp = xgb.XGBRegressor(n_estimators=100, random_state=42)
xg_model_kp.fit(X_train, y_train['Kp'])

# XGBoost Regressor for Ki
xg_model_ki = xgb.XGBRegressor(n_estimators=100, random_state=42)
xg_model_ki.fit(X_train, y_train['Ki'])

# XGBoost Regressor for Kd
xg_model_kd = xgb.XGBRegressor(n_estimators=100, random_state=42)
xg_model_kd.fit(X_train, y_train['Kd'])

### Model Evaluation: R-squared (R²) Score
We will evaluate the performance of each model using the R-squared (R²) score, which indicates the proportion of variance in the target variable that is explained by the model. The higher the R² score, the better the model.


In [ ]:
# Make Predictions for each model
ada_predictions_kp = ada_model_kp.predict(X_test)
ada_predictions_ki = ada_model_ki.predict(X_test)
ada_predictions_kd = ada_model_kd.predict(X_test)

gb_predictions_kp = gb_model_kp.predict(X_test)
gb_predictions_ki = gb_model_ki.predict(X_test)
gb_predictions_kd = gb_model_kd.predict(X_test)

xg_predictions_kp = xg_model_kp.predict(X_test)
xg_predictions_ki = xg_model_ki.predict(X_test)
xg_predictions_kd = xg_model_kd.predict(X_test)

# R-squared (R²) score for each model and target
r2_ada_kp = r2_score(y_test['Kp'], ada_predictions_kp)
r2_ada_ki = r2_score(y_test['Ki'], ada_predictions_ki)
r2_ada_kd = r2_score(y_test['Kd'], ada_predictions_kd)
r2_gb_kp = r2_score(y_test['Kp'], gb_predictions_kp)
r2_gb_ki = r2_score(y_test['Ki'], gb_predictions_ki)
r2_gb_kd = r2_score(y_test['Kd'], gb_predictions_kd)
r2_xg_kp = r2_score(y_test['Kp'], xg_predictions_kp)
r2_xg_ki = r2_score(y_test['Ki'], xg_predictions_ki)
r2_xg_kd = r2_score(y_test['Kd'], xg_predictions_kd)

# Display R² scores for each model and target
print(f'AdaBoost R² for Kp: {r2_ada_kp:.2f}')
print(f'AdaBoost R² for Ki: {r2_ada_ki:.2f}')
print(f'AdaBoost R² for Kd: {r2_ada_kd:.2f}')
print(f'Gradient Boosting R² for Kp: {r2_gb_kp:.2f}')
print(f'Gradient Boosting R² for Ki: {r2_gb_ki:.2f}')
print(f'Gradient Boosting R² for Kd: {r2_gb_kd:.2f}')
print(f'XGBoost R² for Kp: {r2_xg_kp:.2f}')
print(f'XGBoost R² for Ki: {r2_xg_ki:.2f}')
print(f'XGBoost R² for Kd: {r2_xg_kd:.2f}')

### Visualizing Predictions vs True Values
We will now visualize the predictions made by each model against the true values for each of the PID parameters (`Kp`, `Ki`, `Kd`). This will allow us to visually compare the performance of the models.

In [ ]:
# Function to plot predictions vs true values
def plot_predictions(true_values, predicted_values, title):
    plt.scatter(true_values, predicted_values, alpha=0.5)
    plt.plot([min(true_values), max(true_values)], [min(true_values), max(true_values)], color='red', linestyle='--')
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.title(title)
    plt.show()

# Plot for Kp
plt.figure(figsize=(10, 6))
plot_predictions(y_test['Kp'], ada_predictions_kp, 'AdaBoost: Kp Predictions vs True Values')
plt.figure(figsize=(10, 6))
plot_predictions(y_test['Kp'], gb_predictions_kp, 'Gradient Boosting: Kp Predictions vs True Values')
plt.figure(figsize=(10, 6))
plot_predictions(y_test['Kp'], xg_predictions_kp, 'XGBoost: Kp Predictions vs True Values')

# Plot for Ki
plt.figure(figsize=(10, 6))
plot_predictions(y_test['Ki'], ada_predictions_ki, 'AdaBoost: Ki Predictions vs True Values')
plt.figure(figsize=(10, 6))
plot_predictions(y_test['Ki'], gb_predictions_ki, 'Gradient Boosting: Ki Predictions vs True Values')
plt.figure(figsize=(10, 6))
plot_predictions(y_test['Ki'], xg_predictions_ki, 'XGBoost: Ki Predictions vs True Values')

# Plot for Kd
plt.figure(figsize=(10, 6))
plot_predictions(y_test['Kd'], ada_predictions_kd, 'AdaBoost: Kd Predictions vs True Values')
plt.figure(figsize=(10, 6))
plot_predictions(y_test['Kd'], gb_predictions_kd, 'Gradient Boosting: Kd Predictions vs True Values')
plt.figure(figsize=(10, 6))
plot_predictions(y_test['Kd'], xg_predictions_kd, 'XGBoost: Kd Predictions vs True Values')